<a href="https://colab.research.google.com/github/hedayet13/learning-tensorflow/blob/main/transfer_learning_in_tensorflow_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Sat Jul 17 13:59:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

#unzipping

zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2021-07-17 15:27:55--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.13.80, 172.217.13.240, 172.217.15.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.13.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   162MB/s    in 1.0s    

2021-07-17 15:27:56 (162 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [2]:
import os 

for dirpath,dirnames,filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE =(224,224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen= ImageDataGenerator(rescale=1/255.)

print("training Images:")

train_data_10_percent= train_datagen.flow_from_directory(train_dir,
                                                         target_size =IMAGE_SHAPE,
                                                         batch_size = BATCH_SIZE,
                                                         class_mode ="categorical")

print("testing Images")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size= IMAGE_SHAPE,
                                             batch_size = BATCH_SIZE,
                                             class_mode = "categorical")

training Images:
Found 750 images belonging to 10 classes.
testing Images
Found 2500 images belonging to 10 classes.


In [15]:
##setting up callback

# Tensorboard callback

import datetime

def create_tensorboard_callback(dir_name,experiment_name):
  log_dir = dir_name +"/"+experiment_name + "/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.Tensorboard(log_dir = log_dir)
  print(f"saving TnsorBoard log files to : {log_dir}")
  return tensorboard_callback

In [16]:
resnet_url ='https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5'

efficientnet_url = 'https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1'

In [17]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [24]:
def create_model(model_url,num_classes = 10):
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name = "Feature_extraction_layer",
                                           input_shape = IMAGE_SHAPE+(3,))
  model = tf.keras.Sequential(
      [
       feature_extractor_layer,
       tf.keras.layers.Dense(num_classes,activation = "softmax",name="output_layer")
      ]
  )
  return model

In [25]:
resnet_model = create_model(model_url=resnet_url,
                            num_classes = train_data_10_percent.num_classes)

In [27]:
resnet_model.compile(
    loss ="categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [28]:
resnet_history = resnet_model.fit(train_data_10_percent,
                           epochs = 5,
                           steps_per_epoch = len(train_data_10_percent),
                           validation_data = test_data,
                           validation_steps = len(test_data),
                           callbacks = [create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                    experiment_name = "resnet50v2")])

Epoch 1/5
 4/24 [====>.........................] - ETA: 1:29 - loss: 2.6478 - accuracy: 0.1016

KeyboardInterrupt: ignored

In [26]:
resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Feature_extraction_layer (Ke (None, 2048)              23564800  
_________________________________________________________________
output_layer (Dense)         (None, 10)                20490     
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________
